In [17]:
# import packages
import pandas as pd
import pyautogui
# Disable Retina scaling 
pyautogui.useRetina = True
import os
# import cv2

In [18]:
path_task_seq_folder = os.path.abspath(os.path.join(os.getcwd(),'..', 'ref_files','task_sequences'))
path_ref_images = os.path.abspath(os.path.join(os.getcwd(),'..', 'ref_files','ref_images'))

In [19]:
def read_action_sequence(file_name: str):
    """
    Reads the action sequence file and returns it as a pandas DataFrame with correct types.
    """
    global path_task_seq_folder
    file_path = os.path.join(path_task_seq_folder,file_name+'.csv')
    
    df = pd.read_csv(file_path)
    df['sequence_num'] = df['sequence_num'].astype('Int32')
    df['action_type'] = df['action_type'].astype(str)
    df['reference_image'] = df['reference_image'].astype(str)
    df['pre_condition_image'] = df['pre_condition_image'].astype(str)
    df['click_num'] = df['click_num'].astype('Int32')
    df['input_text'] = df['input_text'].astype(str)
    df['key_press'] = df['key_press'].astype(str)
    df['move_to'] = df['move_to'].astype(str)
    df.sort_values(by='sequence_num', ascending=True, inplace=True) # sort dataframe by action sequence

    return df


In [20]:
def check_condition(image_name = ''):
    """
    Check if the condition of action satisfy.
    """
    global path_ref_images
    image_path = os.path.abspath(os.path.join(path_ref_images,image_name+'.png'))
    #print(f'check_condition image_path: {image_path}') #test
    
    print('Check Action Condition...', end=' -> ')
    if image_name == 'nan':
        print(f'No condition required!', end=' -> ')
        return True
    else:
        while True:
            try:
                loc = pyautogui.locateCenterOnScreen(image_path, confidence=0.8)
            except:
                continue
            if loc is not None:
                print(f'Condition - {image_name} Satisfied!', end=' -> ')
                break
        return True

def perform_action(action_type, image_name:str = None, click_num:int = 1, input_text:str = None, key_press:str = None, move_to:str = None):
    """
    Performs the specified action (click, move, keyboard input, key press).
    - Click: 
    - Move:
    - keyboard input: 
    - key press:
    """

    global path_ref_images

    if action_type == "click" and image_name is not None:
        print(f'Locating {image_name} ...', end=' ')
        image_path = os.path.abspath(os.path.join(path_ref_images,image_name+'.png'))
        while True:
            x, y = pyautogui.locateCenterOnScreen(image_path, confidence=0.9)
            if x is not None:
                print('Success!', end=' -> ')
                break
        #test
        print(f'click num {click_num}', end=' -> ')
        pyautogui.click(x/2, y/2, clicks = click_num, interval=0.1, duration = 0.3)
        print(f'Mouse Clicked!')
    elif action_type == "move" and move_to is not None:
        import re
        if bool(re.search(r'\(\d+,\s*\d+\)', move_to)):
            move_to = tuple(map(int, move_to.strip('()').split(",")))
            x = move_to[0]
            y = move_to[1]
            print(f'Move to a corrdinate: ({x}, {y})', end=' -> ')
            pyautogui.moveTo(x, y)
            print(f'Cursor Moved!')
        else:
            # reference a pic
            image_name = move_to

            print(f'Locating {image_name} ...', end=' ')
            image_path = os.path.abspath(os.path.join(path_ref_images,image_name+'.png'))
            while True:
                x, y = pyautogui.locateCenterOnScreen(image_path, confidence=0.8)
                if x is not None:
                    print('Success!', end=' -> ')
                    break
            pyautogui.moveTo(x, y)
            print(f'Cursor Moved!')
    elif action_type == "keyboard_input" and input_text is not None:
        pyautogui.typewrite(input_text)
        print(f'Text Written!')
    elif action_type == "key_press":
        keys = key_press.split(sep='+')
        if len(keys)>1:
            pyautogui.hotkey(keys)
            print(f'Hot Key Pressed!')
        elif len(keys)==1:
            pyautogui.press(keys[0])
            print(f'Key Pressed!')
    else:
        print('Wrong action_type value!')
        exit

In [21]:
def execute_action_sequence(task_seq_file_name, path_task_seq_folder, path_ref_images): 
    """ 
    Executes the action sequence defined in the DataFrame. 
    """ 

    print(f'Executing Task: {task_seq_file_name} ...')
    
    task_sequence_df = read_action_sequence(task_seq_file_name)
    #print(task_sequence_df.dtypes)

    for index, row in task_sequence_df.iterrows(): 
        sequence_num = row['sequence_num']
        action_type = row['action_type'] 
        reference_image = row['reference_image'] 
        pre_condition_image = row['pre_condition_image']
        click_num = row['click_num']
        input_text = row['input_text']
        key_press = row['key_press']
        move_to = row['move_to']
        
        while True:
            print(f'\tStep No.{sequence_num}', end=': ')
            if check_condition(pre_condition_image):
                perform_action(action_type, reference_image, click_num, input_text, key_press, move_to)
                break

In [22]:
execute_action_sequence('task1', path_task_seq_folder, path_ref_images)

Executing Task: task1 ...
	Step No.1: Check Action Condition... -> Condition - excel_icon Satisfied! -> Locating excel_icon ... Success! -> click num 1 -> Mouse Clicked!
	Step No.2: Check Action Condition... -> Condition - excel_initial_window Satisfied! -> Locating new_blank_workbook ... Success! -> click num 2 -> Mouse Clicked!
	Step No.3: Check Action Condition... -> No condition required! -> Locating create ... Success! -> click num 1 -> Mouse Clicked!
	Step No.4: Check Action Condition... -> Condition - excel_window Satisfied! -> Locating upper_space ... Success! -> click num 2 -> Mouse Clicked!
	Step No.5: Check Action Condition... -> Condition - active_cell Satisfied! -> Text Written!
	Step No.6: Check Action Condition... -> No condition required! -> Key Pressed!
	Step No.7: Check Action Condition... -> No condition required! -> Text Written!
	Step No.8: Check Action Condition... -> No condition required! -> Locating save_button ... Success! -> click num 1 -> Mouse Clicked!
	Ste

# TEST

In [23]:
screen_width, screen_height = pyautogui.size()
print(f"Screen width: {screen_width}, Screen height: {screen_height}")

Screen width: 1512, Screen height: 982


In [28]:
scenario_handlers = {
        'save_window': {
            'replace_file': {
                'click': {
                    'image': 'replace_yes',  # Image to click when file exists
                    'clicks': 1
                }
            }
        },
        # Add more scenario handlers as needed
    }
pre_condition_image = 'save_window'

print(scenario_handlers.get(pre_condition_image, {}))


{'replace_file': {'click': {'image': 'replace_yes', 'clicks': 1}}}
